In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

def preprocess_data(folder_path, img_size=(224, 224)):
    """
    Preprocesses image data from a folder containing subfolders for each class.

    Parameters:
    - folder_path: Path to the root folder containing subfolders for each class.
    - img_size: Tuple, size to resize the images to (default is (224, 224)).

    Returns:
    - X: NumPy array containing the preprocessed images.
    - y: NumPy array containing the labels.
    """

    images = []
    labels = []

    for i, class_name in enumerate(os.listdir(folder_path)):
        class_folder = os.path.join(folder_path, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path)

            # Resize image
            img = cv2.resize(img, img_size)

            # Convert to grayscale
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Normalize pixel values (0-1)
            img = img.astype('float32') / 255.0

            images.append(img)
            labels.append(i)  # Assign a label based on folder index

    X = np.array(images)
    y = np.array(labels)

    return X, y

# Example folder structure: 
# - Root folder
#    - autism (contains images of autism children)
#    - normal (contains images of normal children)

root_folder = "datasets/autism"  # Update with your root folder path
img_size = (224, 224)

# Preprocess data
X, y = preprocess_data(root_folder, img_size)

# Shuffle the data
X, y = shuffle(X, y, random_state=42)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for CNN input (add channel dimension for grayscale)
X_train = X_train.reshape(X_train.shape[0], img_size[0], img_size[1], 1)
X_test = X_test.reshape(X_test.shape[0], img_size[0], img_size[1], 1)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Flatten the training and testing data
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Train SVM classifier
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(X_train_flat, y_train)

# Make predictions
train_predictions_image = svm_classifier.predict(X_train_flat)
test_predictions_image = svm_classifier.predict(X_test_flat)

# Evaluate model
train_accuracy = accuracy_score(y_train, train_predictions_image)
test_accuracy = accuracy_score(y_test, test_predictions_image)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Print classification report
print("\nTraining Classification Report:")
print(classification_report(y_train, train_predictions_image))

print("\nTesting Classification Report:")
print(classification_report(y_test, test_predictions_image))

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
data = pd.read_csv("datasets/autism.csv")
mean=data["Age"].mean()
mod=data["Understand facial expressions and body language?"].mode()[0]
data["Age"].fillna(mean, inplace=True)
data["Understand facial expressions and body language?"].fillna(mod, inplace=True)
outliers = (data['Age'] < 7) | (data['Age'] > 20)

mean_age = data.loc[~outliers, 'Age'].mean()  # Calculate mean excluding outliers
print(mean)
data.loc[outliers, 'Age'] = mean_age
columns_to_drop = ['Name', 'Parent', 'Address']
data = data.drop(columns=columns_to_drop, axis=1)
label_encoder = LabelEncoder()
data["Gender"] = label_encoder.fit_transform(data["Gender"])
data["Autism?"] = label_encoder.fit_transform(data["Autism?"])
data["Physical problem?"] = label_encoder.fit_transform(data["Physical problem?"])
data = pd.get_dummies(data, columns=['Understand facial expressions and body language?'], prefix='', prefix_sep='')
data["Body"] = label_encoder.fit_transform(data["body"])
data["Facial"] = label_encoder.fit_transform(data["face"])
data["Facial"] = label_encoder.fit_transform(data["both"])
data.drop(columns="none", inplace=True)
y = data["Autism?"]  # Feature Vector
X = data.drop("Autism?",axis=1)  # Feature Matrix
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Scale all columns
X = scaler.fit_transform(X)
from sklearn.model_selection import train_test_split


# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train the SVM classifier
# model = SVC(kernel='linear')
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

# Make predictions on the testing set
predictions_csv = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions_csv)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, predictions_csv))